In [1]:
import numpy as np
import torch
from models import StainNet, ResnetGenerator
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import os

In [2]:
def norm(image):
    image = np.array(image).astype(np.float32)
    image = image.transpose((2, 0, 1))
    image = ((image / 255) - 0.5) / 0.5
    image=image[np.newaxis, ...]
    image=torch.from_numpy(image)
    return image

def un_norm(image):
    image = image.cpu().detach().numpy()[0]
    image = ((image * 0.5 + 0.5) * 255).astype(np.uint8).transpose((1,2,0))
    return image

In [3]:
model_Net = StainNet().cuda()
model_Net.load_state_dict(torch.load(r"C:\Users\Kevin\PycharmProjects\wsi_analysis\kevin\he2ihc\preprocessing\StainNet-Public_layer3_ch32.pth"))
model_Net.eval()

StainNet(
  (rgb_trans): Sequential(
    (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [4]:
he_src = r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\he2ihc_tiles\HE"
he_img_list = [os.path.join(he_src,x) for x in os.listdir(he_src) if x.endswith(".png")]

In [5]:
for he_img_list in tqdm(he_img_list,colour="red",desc="Images processed:"):
    with torch.no_grad():
        filename = os.path.basename(he_img_list)
        save_path = os.path.join(r"\\10.99.68.178\ashleyex\Type_1_diabetes\IHC to HE model\he2ihc_tiles\HE\stain_normalization",filename)
        source_img = cv2.imread(he_img_list, cv2.COLOR_BGR2RGB)
        img_net=model_Net(norm(source_img).cuda())
        img_net=un_norm(img_net)
        cv2.imwrite(save_path,img_net)

Images processed:: 100%|██████████| 1466/1466 [20:16<00:00,  1.21it/s]
